# Financials Verkennende Analyse

In [41]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, BooleanType
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import when, sum, col, concat_ws
from pyspark.sql.functions import udf,explode, map_keys

spark = SparkSession.builder.getOrCreate()

In [42]:
df1 = spark.read.load('/mnt/prepared/SupplyChain/StoreOrder/StoreOrderLine')
df1.show(5)

+-------------------+-------+-----------+----------------+-------------------+------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|         InsertDate|StoreNo|WarehouseNo|DeliveryStreamNo|   DeliveryDateTime|StoreOrderNo| Soino|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|      date|year|month|day|
+-------------------+-------+-----------+----------------+-------------------+------------+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+----------+----+-----+---+
|2015-11-26 01:00:00|   4077|        900|              96|2015-11-23 20:00:00|    50363056|295606|       1|       null|           N|                  N|              null|    872606|              N|                N|2015-11-23|2015|   11| 23|
|2015-11-26 01:00:00|   1645

In [43]:
df2 = spark.read.load('dbfs:/mnt/thorcf2/clean/supp/dedi/storeorder')
df2_flatten = df2.select("Header.*", "Payload.*", "DeliveryDatePartitionKey", "__meta.*")
df2_flatten_2 = df2_flatten.select("Event.*", "StoreOrder.*", "DeliveryDatePartitionKey", "ModifiedTimestamp")
df2_flatten_3 = df2_flatten_2.select("Action", "DateTime", "Opco.*", "StoreNo", "WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod","StoreOrderLine","DeliveryDatePartitionKey","ModifiedTimestamp")

In [44]:
# df2_flatten_3.select("StoreOrderLine").show()

In [45]:
df2_flatten_3.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- StoreOrderLine: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- SoiNo: long (nullable = true)
 |    |    |-- Quantity: integer (nullable = true)
 |    |    |-- OrderLineId: integer (nullable = true)
 |    |    |-- PromotionInd: string (nullable = true)
 |    |    |-- InitialPromotionInd: string (nullable = true)
 |    |    |-- SupplierCompanyGln: long (nullable = true)
 |    |    |-- SupplierNo: integer (nullable =

In [46]:
print(f"df2: {df2.count()} {len(df2.columns)}")
# print(f"df2_flatten: {df2_flatten.count()} {len(df2_flatten.columns)}")
# print(f"df2_flatten_2: {df2_flatten_2.count()} {len(df2_flatten_2.columns)}")
# print(f"df2_flatten_3: {df2_flatten_3.count()} {len(df2_flatten_3.columns)}")
# print(f"df1: {df1.count()} {len(df1.columns)}")

In [47]:
df_storeorderline = df2_flatten_3.select("StoreOrderLine")
df_storeorderline.printSchema()
df_storeorderline.show(1)

root
 |-- StoreOrderLine: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- SoiNo: long (nullable = true)
 |    |    |-- Quantity: integer (nullable = true)
 |    |    |-- OrderLineId: integer (nullable = true)
 |    |    |-- PromotionInd: string (nullable = true)
 |    |    |-- InitialPromotionInd: string (nullable = true)
 |    |    |-- SupplierCompanyGln: long (nullable = true)
 |    |    |-- SupplierNo: integer (nullable = true)
 |    |    |-- AddUnorderedInd: string (nullable = true)
 |    |    |-- EmergencyOrderInd: string (nullable = true)

+--------------------+
|      StoreOrderLine|
+--------------------+
|[{3915, 1, 1, N, ...|
+--------------------+
only showing top 1 row



In [60]:
df = df_storeorderline.select(*[F.col('StoreOrderLine')[i].alias(f'StoreOrderLine{i}') for i in range(1)])
df.show()

+--------------------+
|     StoreOrderLine0|
+--------------------+
|{3915, 1, 1, N, N...|
|{424625, 1, 742, ...|
|{195987, 1, 352, ...|
|{11314, 1, 1, N, ...|
|{12425, 1, 1, N, ...|
|{405591, 27, 275,...|
|{410104, 1, 364, ...|
|{445250, 1, 226, ...|
|{1260, 1, 1, N, N...|
|{506663, 3, 95, N...|
|{23429, 1, 355, N...|
|{347612, 1, 49, N...|
|{3915, 2, 1, N, N...|
|{12256, 1, 1, N, ...|
|{62224, 1, 1, N, ...|
|{3915, 1, 1, N, N...|
|{255485, 1, 980, ...|
|{230224, 1, 12, N...|
|{353621, 1, 126, ...|
|{269143, 1, 22, Y...|
+--------------------+
only showing top 20 rows



In [61]:
df.printSchema()

root
 |-- StoreOrderLine0: struct (nullable = true)
 |    |-- SoiNo: long (nullable = true)
 |    |-- Quantity: integer (nullable = true)
 |    |-- OrderLineId: integer (nullable = true)
 |    |-- PromotionInd: string (nullable = true)
 |    |-- InitialPromotionInd: string (nullable = true)
 |    |-- SupplierCompanyGln: long (nullable = true)
 |    |-- SupplierNo: integer (nullable = true)
 |    |-- AddUnorderedInd: string (nullable = true)
 |    |-- EmergencyOrderInd: string (nullable = true)



In [63]:
df_flatten = df.select('StoreOrderLine0.*')
df_flatten.show()

+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+
| SoiNo|Quantity|OrderLineId|PromotionInd|InitialPromotionInd|SupplierCompanyGln|SupplierNo|AddUnorderedInd|EmergencyOrderInd|
+------+--------+-----------+------------+-------------------+------------------+----------+---------------+-----------------+
|  3915|       1|          1|           N|                  N|     8714253049021|    669143|              N|                N|
|424625|       1|        742|           N|                  N|     8710508900000|    320838|              N|                N|
|195987|       1|        352|           N|                  N|     5410091300074|    326553|              N|                N|
| 11314|       1|          1|           N|                  N|     8710398000002|    778225|              N|                N|
| 12425|       1|          1|           N|                  N|     8710646000013|     32755|              N|   